<a href="https://colab.research.google.com/github/hannari-python/tutorial/blob/master/household/kakei_chosa_prepro_to_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 家計調査データの分析

## データの紹介

- [独立行政法人 統計センター 一般用ミクロデータ 平成21年全国消費実態調査 （十大費目）](https://github.com/hannari-python/tutorial/raw/master/data/ippan-microdata/ippan_2009zensho.zip) https://www.nstac.go.jp/services/ippan-microdata.html
- [独立行政法人 統計センター 一般用ミクロデータ 平成21年全国消費実態調査 （詳細品目）](https://github.com/hannari-python/tutorial/raw/master/data/ippan-microdata/ippan_2009zensho_s.zip) https://www.nstac.go.jp/services/ippan-microdata.html
- [独立行政法人 統計センター 一般用ミクロデータ 就業構造基本調査 （平成４年～24年）](https://github.com/hannari-python/tutorial/raw/master/data/ippan-microdata/ippan_shugyou.zip) https://www.nstac.go.jp/services/ippan-microdata.html

## 課題1

https://www.nstac.go.jp/services/ippan-microdata.html

をクリックし、ippan_2009zensho.zip と ippan_2009zensho_s.zip がどのように異なるか見てみましょう。

## データの取得と解凍

! は ...と呼ばれます。
!の後にLinuxコマンドを書くとそのセルの命令はPythonプログラムではなくLinuxのコマンドとして実行されます。

wget はデータをダウンロードするのに使うことができるLinuxのコマンドです。

In [ ]:
!wget https://github.com/hannari-python/tutorial/raw/master/data/ippan-microdata/ippan_2009zensho_s.zip

unzip を zip ファイルを解凍する Linux コマンドです。

In [ ]:
!unzip /content/ippan_2009zensho_s.zip

## 課題2

- `ippan_2009zensho_s_dataset.csv` の何行目からが表データのスタートか見てみましょう


## 分析に必要なPythonパッケージのインストール

pip はPythonのパッケージをインストールするためのコマンドです。
空白を間にはさみ複数のパッケージを連続して書くとそれらがすべてインストールされます。
`-U` オプションは ... を意味します。

dashは...

jupyter_dashは...

plotlyは

を行うPythonパッケージです。

In [ ]:
!pip install dash jupyter_dash 
!pip install -U plotly

In [ ]:
import pandas as pd

In [ ]:
s = pd.read_csv('/content/ippan_2009zensho_s/ippan_2009zensho_s_dataset.csv', encoding='shift_jis', header=8)

## 課題3
- `encoding` オプションを付けないと`s`がどうなるか確認しましょう
- `header` オプションを付けないと`s`がどうなるか確認しましょう
- `header` オプションの数値を変えると`s`がどうなるか確認しましょう

In [ ]:
s.shape

## データフレームとは

ここで `s` のデータ構造はデータフレームと呼ばれます。
データフレームとは...

## 課題4

- `ippan_2009zensho_s.xls` を Google Sheets で開いて、データ行列の定義を確認しましょう。

## データフレームの列の選択

`s` のうち ... の列を選択し ... だけを取得してみましょう
そのためにはデータフレームの `loc` メソッドを使います。

In [ ]:
data = s.loc[:, :'OL_Expenditure']
data

## [記述統計量(英: descriptive statistics value)](https://ja.wikipedia.org/wiki/%E8%A6%81%E7%B4%84%E7%B5%B1%E8%A8%88%E9%87%8F) の確認

記述統計量を確認しデータの範囲や代表的な値がどの程度か確認しましょう。
そのためにはデータフレームの `describe` メソッドを使います。


In [ ]:
data.describe()

## plotly (express) を用いた可視化

plotlyとは ... です。
ここでは ... を可視化してみましょう。

In [ ]:
import plotly.express as px

In [ ]:
px.violin(data, y=data.columns)

In [ ]:
px.histogram(data, x='Y_Income', histnorm='percent', nbins=100)

In [ ]:
data_bins = pd.cut(data['Y_Income'].values, bins=10)

In [ ]:
data_bins

# ↑いまいちなビン作り・・・
# ↓自作bin作成

In [ ]:
bin_image = [0, 2000, 4000, 6000, 8000, 12000, 20000]

In [ ]:
bin_image

In [ ]:
import numpy as np

In [ ]:
bin_image = [0, 2000, 4000, 6000, 8000, 12000, 20000]
bin_array = np.digitize(data['Y_Income'], bin_image)

In [ ]:
cont = pd.read_excel("/content/ippan_2009zensho_s/ippan_2009zensho_s.xls")

In [ ]:
cont[:50]

## 各消費分類の消費支出に対する割合を作成する

In [ ]:
data_test = data.loc[: ,'L_Expenditure':'OL_Expenditure']

In [ ]:
data_test

In [ ]:
data_test.apply(lambda x: x/ data_test['L_Expenditure'])

In [ ]:
data_expenditure_ratio = data_test.apply(lambda x: x / data_test['L_Expenditure'])
data_front = data.loc[:, :'Y_Income']

In [ ]:
px.violin(data_expenditure_ratio, x=data_expenditure_ratio.columns[1:])

In [ ]:
data_front = data_front.drop('Weight',axis=1)

In [ ]:
data_preped = pd.concat([data_front, data_expenditure_ratio], axis=1)

In [ ]:
data_preped

In [ ]:
bin_image = [0, 2000, 4000, 6000, 8000, 12000, 20000]
bin_array = np.digitize(data['Y_Income'], bin_image)
data_preped['bins'] = bin_array

In [ ]:
data_preped

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()

for num in data_preped.bins.unique():
  data_preped_num = data_preped[data_preped['bins'] == num]
  fig.add_trace(go.Histogram(x=data_preped_num['Food'], name=f'{num}', histnorm='probability', nbinsx=20))

fig.show()

## どんな可視化をするか？

- ビンの個数を動かせる
- ビンの数値を変更できる
- それをヒストグラムで描画できる

In [ ]:
data_for_dash = data_preped.drop('bins', axis=1)

In [ ]:
from jupyter_dash import JupyterDash 
import dash_core_components as dcc 
import dash_html_components as html 

from dash.dependencies import Input, Output, State, ALL

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
                       

                       html.Button(id='my_button', children='Add Input'),
                      
                      html.Div([
                       html.Div([
                       html.Div(id='input_zone', children=[]),
                      ], style={'width': '25%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                       


                       html.Div([
                                 dcc.Dropdown(id='my_dropdown',
                                              options=[{'label': col, 'value': col} for col in data_for_dash.columns],
                                              value='Food'
                                              ),
                                 dcc.Graph(id='my_graph'),
                                
                                html.Div([
                                 html.H3('Histogram Bin Num: '),
                                 dcc.Input(id='bin_num', value=10, type='number'),
                                ]),
                       ], style={'width': '70%', 'display': 'inline-block'}),
                      ]),
                       
                       html.Div([
                       dcc.RangeSlider(id='my_range_slider',
                                       min=0,
                                       max=data_for_dash['Y_Income'].max()
                                       
                                       ),
                        html.Button(id='slider_button', children='graph update'),
                       ], style={'width': '90%', 'height': 100, 'margin': 'auto'}),

])

@app.callback(Output('input_zone', 'children'), [Input('my_button', 'n_clicks')], [State('input_zone', 'children')], prevent_initial_call=True)
def update_input_zone(n_clicks, existing_children):
  my_inputs = html.Div([
                        dcc.Input(id={'type': 'my_inputs', 'index': n_clicks}, value=0)
  ])
  existing_children.append(my_inputs)
  return existing_children

@app.callback(Output('my_range_slider', 'value'), [Input({'type': 'my_inputs', 'index': ALL}, 'value')])
def update_slider(enter_values):
  if len(enter_values) > 1:
    enter_values = [int(i) for i in enter_values]
  return enter_values

@app.callback(Output('my_graph', 'figure'), [Input('slider_button', 'n_clicks'), Input('my_dropdown', 'value'), Input('bin_num', 'value')], [State('my_range_slider','value')], prevent_initial_call=True)
def update_graph(n_clicks, slider_values, bin_num, selected_values):

  if len(selected_values) > 1 and sum(selected_values) > 1:
    bin_array = np.digitize(data_for_dash['Y_Income'], selected_values)
    data_for_dash['bins'] = bin_array  
    fig = go.Figure()
    for num in data_for_dash['bins'].unique():
      update_df = data_for_dash[data_for_dash['bins'] == num]
      fig.add_trace(go.Histogram(x=update_df[slider_values], histnorm='probability', nbinsx=bin_num))
  
    return fig
  return dash.no_update


app.run_server(mode='inline')

In [ ]:
data_for_dash

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir '/content/drive/My Drive/pycon-tutorial'

In [ ]:
data_for_dash.to_csv('/content/drive/My Drive/pycon-tutorial/data_for_dash.csv')

In [ ]:
test = dcc.RangeSlider(min=0, max=100, value=[0,100,200,300,400])

In [ ]:
test.value

In [ ]:
sum(test.value)

In [ ]:
len(test.value)

In [ ]:
if (len(test.value) > 4) & (sum(test.value) > 1):
  print("yeah")
else:
  print('nooo')